In [13]:
import pandas as pd
import re
import difflib
from difflib import get_close_matches as gcm

skills = pd.read_csv('skills_db/skill.csv')
skills.head()

,skill_id,skill
0,415,Applied Science
1,473,Arts
2,988,Business
3,1634,Computer Science
4,2099,Data Science


In [2]:
sk_list = skills['skill'].tolist()
len(sk_list)

9134

In [3]:
test = [
    "This is an example job description for a computer science role. Requirement: Python, C, Java",
    "This is another example of job description for a marketing related role. Soft skills and marketing skills required.",
    "Data science job. ML, AI, DeepLearning"
]

In [6]:
%time
for t in test:
    print([s for s in sk_list if t.lower().find(s.lower()) > -1])

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs
['Computer Science', 'B', 'C', 'D', 'Java', 'PL', 'Python', 'R']
['Marketing', 'B', 'C', 'D', 'PL', 'R', 'Market']
['Data Science', 'B', 'C', 'D', 'ML', 'PL', 'R', 'Data', 'Learning']


In [8]:
def similarity(word, pattern):
    return difflib.SequenceMatcher(a=word.lower(), b=pattern.lower()).ratio()

threshold = 0.9

In [ ]:


keywords = ['Projekt-Nr.:', 'Projektbezeichnung:', 'Anlagenklassifizierung:', 'Arbeiten / Gewerk:']
unk_text = "Projekthezeichnung: —_[H- Kloster Eig i Krankenhaus"
words = unk_text.split()

result = [gcm(kw, words, n=len(words), cutoff=0.8) for kw in keywords]
# [[], ['Projekthezeichnung:'], [], []]

In [24]:
%time
print()
for t in test:
    print(t)
    words = t.split()
    results = []
    for s in sk_list:
        if '(' in s:
            abb = s[s.find("(")+1:s.find(")")]
            result = gcm(abb, words, n=len(words), cutoff=0.9)
            if len(result) > 0:
                results.append(s)
                continue
        result = gcm(s, words, n=len(words), cutoff=0.8)
        if len(result) > 0:
            results.append(s)
    print(results)
    print()

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs

This is an example job description for a computer science role. Requirement: Python, C, Java
['Java', 'Python', 'IPython', 'Retirement', 'Man', 'Conscience', 'Uan', 'Interface (Java)', 'Javac', 'Javafx', 'Idle (Python)']

This is another example of job description for a marketing related role. Soft skills and marketing skills required.
['Marketing', 'Hand']

Data science job. ML, AI, DeepLearning
['Deep Learning', 'ML', 'Data', 'Conscience', 'Learning']



In [12]:
%time
print()
for t in test:
    print(t)
    words = t.split()
    for s in sk_list:
        if similarity(s, t) > 0.5:
            print(s, end=';')
    print()

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 5.25 µs
This is an example job description for a computer science role. Requirement: Python, C, Java

This is another example of job description for a marketing related role. Soft skills and marketing skills required.

Data science job. ML, AI, DeepLearning
Machine Learning;Data Engineering;Data Science Tool;Temporal Difference (TD) Learning;Data Encoding;Inductive Machine Learning;Applied Machine Learning;Temporal Difference Learning;


In [ ]:
import difflib
text = "Here is the text we are trying to match across to find the three word
        sequence n0 inf0rmation available I wonder if we will find it?"    
words = text.split()
three = [' '.join([i,j,k]) for i,j,k in zip(words, words[1:], words[2:])]
print difflib.get_close_matches('no information available', three, cutoff=0.9)
#Oyutput:
['n0 inf0rmation available']